## FineTuning Tinyllama 

This notebook aims to look at whether Tinyllama model can be finetuned using Qlora to improve results.

In [1]:
import os
import pickle
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

C:\Users\utkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


### Idea 1: Fine Tune on Successful conversations from other models

Use the 4 models who are have had successful conversations and use their conversations as data to finetune model.

In [2]:
# Read the Conversations to be Trained on:
paths = ["./evaluation_results/mistral_instruct3_convo.pkl",
         "./evaluation_results/mistral_instruct4_convo.pkl",
         "./evaluation_results/llama4_convo.pkl",
        './evaluation_results/chatgpt_convo.pkl']

conversations = []
for path in paths:
    with open(path, 'rb') as file:
        conversation = pickle.load(file)
        conversations.append(conversation)

#### Convert each conversations into Tiny Llama Chat Template

<|system|>  
Write system here</s>  
<|user|>  
Write user response here</s>   
<|assistant|>

In [3]:
# Convert Mistral to Tinyllama Template

def mistral_to_tiny(convo):
    prompt = convo[1].replace("<s> [INST] ", "<|user|>\n").replace("</s>  [INST] ", "</s> \n<|user|>\n")
    prompt = prompt.replace(" [/INST] ", "</s> \n<|assistant|>\n").replace(" [/INST]", "</s>")
    return [prompt, convo[2]]

conversations[0] = [mistral_to_tiny(convo) for convo in conversations[0]]
conversations[1] = [mistral_to_tiny(convo) for convo in conversations[1]]

In [4]:
# Convert Llama to Tinyllama Template

def llama_to_tiny(convo):
    prompt = convo[1].replace("<|im_start|>system", "<|system|>").replace("<|im_end|>", "</s> ")
    prompt = prompt.replace("<|im_start|>user", "<|user|>").replace("<|im_start|>assistant", "<|assistant|>")
    return [prompt, convo[2]]

conversations[2] = [llama_to_tiny(convo) for convo in conversations[2]]

In [5]:
# Convert ChatGPT to Tinyllama Template

def chatgpt_to_tiny(convo):
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
    tokenized_chat = tokenizer.apply_chat_template(convo[1], tokenize=True, add_generation_prompt=True, return_tensors="pt")
    return [tokenizer.decode(tokenized_chat[0]), convo[2]]

conversations[3] = [chatgpt_to_tiny(convo) for convo in conversations[3]]

In [6]:
combined_convos = []
for convo in conversations:
    combined_convos += convo

In [29]:
aa = ""
for x in combined_convos:
    aa += "<START>" + generate_prompt(x)

In [30]:
aa

'<START><|user|>\nI am the game manager, I will give you a role in a game. You will be speaking to the player after this. I will give you your instructions now and you are to follow them when speaking to the player.You are a game character introducing the player to the game world. The game\'s name is \'Welcome to Learning with AI!\' and the player will be learning about Model Deployment today! Your job will be to introduce the player to the game while asking the following questions: 1. Ask for the player\'s name. 2. Ask the player to give you a name3.: Ask the player to give you a personality for yourself. This can be anything funny and entertaining. You can give a suggestions to the user. 4. Now you just wish to know if the user wants to enable monsters in the game. A yes or no answer would suffice. Ask the questions one by one. Wait for the user to respond to each question first. Do not as next question until  you are satisfied you have gotten the answer to your previous question. On

In [7]:
data = Dataset.from_dict({"Context": [convo[0] for convo in combined_convos],
                          "Response": [convo[1] for convo in combined_convos]})

In [25]:
data

Dataset({
    features: ['Context', 'Response'],
    num_rows: 74
})

#### Create Dataset

In [27]:
def generate_prompt(data_point):
    return data_point[0] + " \n<|assistant|>\n" + data_point[1] + "</s>"


def generate_and_tokenize_prompt(data_point, checkpoint = "HuggingFaceH4/zephyr-7b-beta"):
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    tokenizer.pad_token = tokenizer.eos_token
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True)
    return tokenized_full_prompt

In [9]:
dataset = data.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

#### FineTune TIme

In [23]:
model_path = "jeff31415/TinyLlama-1.1B-1T-OpenOrca"

In [24]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

C:\Users\utkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\utkar\.cache\huggingface\hub\models--jeff31415--TinyLlama-1.1B-1T-OpenOrca. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: No GPU found. A GPU is needed for quantization.

In [14]:
!cd

C:\Users\utkar\Desktop\NUS\FYP\GamificationUsingLLM\game
